**POS-Tagging**

Welcome to the third lab! In this excercise you will build a simple pos-tagger.
The excercise is inspired from Pytorch tutorial site: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

In [1]:
!pip install pandas

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)




tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward>))


**Task:**

Load the `training_data` from `corpus-small.train`, and modify the `tag_to_ix` dictionary to have all different tags.
Build `ix_to_tag` dictionay, you will need it at the last task. 

In [68]:
import pandas as pd
def prepare_sequence(seq, to_ix):  
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

file = open("corpus-small.train", "r")
handle = file.read()
#print(handle)
sentences = list(filter(None, handle.replace('\n',' ').replace(',/,','').split("./.") ))
file.close()
training_data = []
tag_to_ix = {}
word_to_ix = {}
ix_to_tag = {}
for sentence in sentences:
    sentence = sentence.split(" ")
    words = []
    tags = []
    for idx in range(len(sentence)):
        if(len(sentence[idx].rsplit("/",1)) < 2):
            continue
        word = sentence[idx].rsplit("/",1)[0]
        tag = sentence[idx].rsplit("/",1)[1]
        words.append(word)
        tags.append(tag)
        if(tag not in tag_to_ix):
            tag_to_ix[tag] = len(tag_to_ix)
            ix_to_tag[len(tag_to_ix)] = tag
        if(word not in word_to_ix):
            word_to_ix[word] = len(word_to_ix)
    training_data.append((words, tags))
      
# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

In [69]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

### Reduce number of epochs, if training data is big
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        if(len(sentence) == 0 or len(tags) ==0):
            continue
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
import numpy as np
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-3.2468, -3.3564, -3.8260,  ..., -4.0830, -4.0528, -3.9723],
        [-3.4220, -3.3755, -3.8209,  ..., -4.0289, -3.9756, -3.7772],
        [-3.3709, -3.2732, -3.8410,  ..., -4.2046, -4.0161, -4.0316],
        ...,
        [-3.4328, -3.3287, -3.8171,  ..., -4.0190, -4.0555, -3.8396],
        [-3.3939, -3.4040, -3.8380,  ..., -4.0085, -4.0768, -3.8339],
        [-3.4172, -3.4259, -3.8133,  ..., -4.0110, -4.0304, -3.7948]])


**Task:**

Read the test data `corpus-small.test` and process it then get the predicitions.Write down the output tagged predicitons in file `corpus-small.out` in the same form as `corpus-small.answer`.

Note: in-case of unseen word in the testing dataset, replace it with a random seen one! (There's a better solution).

At the end, run the last cell, to get the accuracy of your model.

In [ ]:
%run tagger_eval.py corpus-small.out corpus-small.answer